In [1]:
from pyspark.sql import Window
from pyspark.sql.window import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALSModel

In [3]:
import findspark

# findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/04/12 01:58:17 WARN Utils: Your hostname, DESKTOP-HJGMIR0 resolves to a loopback address: 127.0.1.1; using 172.24.176.133 instead (on interface eth0)
23/04/12 01:58:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/12 01:58:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Test recommender

In [5]:
als_model = ALSModel.load("../data/save")

In [9]:
userid = [ [15]]

useridColumns = StructType([StructField("userid", IntegerType())])
deptDF = spark.createDataFrame(data=userid, schema = useridColumns)

userSubsetRecs = als_model.recommendForUserSubset(deptDF, 5)
nrecommendations = userSubsetRecs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userid', col("rec_exp.businessid"), col("rec_exp.rating"))

nrecommendations.show()

+------+----------+---------+
|userid|businessid|   rating|
+------+----------+---------+
|    15|    145879| 5.664754|
|    15|     49206| 5.655204|
|    15|     62843|5.6471815|
|    15|    146680|5.3232384|
|    15|    140227|5.2772846|
+------+----------+---------+



## Sample not use

In [7]:
business = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_business.json")
review = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_review.json")
user = spark.read.json("../../temp/yelp_json_yelp_academic_dataset_user.json")

In [17]:
userRatings=review.groupBy("user_id").count()
window = Window.orderBy(col('user_id'))
userRatings = userRatings.withColumn('userid', row_number().over(window))
buiRatings=review.groupBy("business_id").count()
window = Window.orderBy(col('business_id'))
buiRatings = buiRatings.withColumn('businessid', row_number().over(window))
newratings=review.join(userRatings, ['user_id'])
newratings=newratings.join(buiRatings, ['business_id'])
newratings=newratings.withColumn("stars",col("stars").cast(IntegerType()))
newratings=newratings.withColumn("date",to_timestamp("date"))

In [19]:
model=ALSModel.load("D:\Movie_Recommendation\data\save")

In [35]:
userid = [ [2]]

useridColumns = StructType([StructField("userid", IntegerType())])
deptDF = spark.createDataFrame(data=userid, schema = useridColumns)

userSubsetRecs = model.recommendForUserSubset(deptDF, 5)
nrecommendations = userSubsetRecs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userid', col("rec_exp.businessid"), col("rec_exp.rating"))

nrecommendations.show()

+------+--------------------+-------------------+
|userid|     recommendations|            rec_exp|
+------+--------------------+-------------------+
|     2|[{35182, 7.335038...| {35182, 7.3350387}|
|     2|[{35182, 7.335038...|{140227, 7.2798605}|
|     2|[{35182, 7.335038...|  {62843, 7.273821}|
|     2|[{35182, 7.335038...| {46174, 7.1696444}|
|     2|[{35182, 7.335038...| {102258, 6.989647}|
+------+--------------------+-------------------+

